In [24]:
 # Instalar o java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [6]:
# Baixar a versao do spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [23]:
!apt-get install oracle-java11-installer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package oracle-java11-installer is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'oracle-java11-installer' has no installation candidate


In [7]:
# deszipar
!tar xf /content/spark-3.5.0-bin-hadoop3.tgz

In [28]:
# criar variaveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [9]:
# instalar a lib findspark para localizar o spark no sistema
!pip install findspark

In [26]:
import findspark
findspark.init()

In [29]:
# criar spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [75]:
from google.colab import files
arquivo = files.upload()

Saving reviews-2.csv to reviews-2.csv


In [32]:
# criar dataframe
# inferSchema serve para fazer inferencia sobre os tipos de dados
df = spark.read.csv('movies.csv', header=True, inferSchema=True)

In [34]:
df.printSchema()

root
 |-- Film: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Lead Studio: string (nullable = true)
 |-- Audience score %: integer (nullable = true)
 |-- Profitability: double (nullable = true)
 |-- Rotten Tomatoes %: integer (nullable = true)
 |-- Worldwide Gross: string (nullable = true)
 |-- Year: integer (nullable = true)



In [36]:
df.show()

+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|                Film|    Genre|         Lead Studio|Audience score %|Profitability|Rotten Tomatoes %|Worldwide Gross|Year|
+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|Zack and Miri Mak...|  Romance|The Weinstein Com...|              70|  1.747541667|               64|        $41.94 |2008|
|     Youth in Revolt|   Comedy|The Weinstein Com...|              52|         1.09|               68|        $19.62 |2010|
|You Will Meet a T...|   Comedy|         Independent|              35|  1.211818182|               43|        $26.66 |2010|
|        When in Rome|   Comedy|              Disney|              44|          0.0|               15|        $43.04 |2010|
|What Happens in V...|   Comedy|                 Fox|              72|  6.267647029|               28|       $219.37 |2008|
| Water 

In [40]:
# selecionar colunas
df_movie = df.select('Film', 'Audience score %', 'Genre', 'Year', 'Lead Studio')


In [41]:
df_movie.show()

+--------------------+----------------+---------+----+--------------------+
|                Film|Audience score %|    Genre|Year|         Lead Studio|
+--------------------+----------------+---------+----+--------------------+
|Zack and Miri Mak...|              70|  Romance|2008|The Weinstein Com...|
|     Youth in Revolt|              52|   Comedy|2010|The Weinstein Com...|
|You Will Meet a T...|              35|   Comedy|2010|         Independent|
|        When in Rome|              44|   Comedy|2010|              Disney|
|What Happens in V...|              72|   Comedy|2008|                 Fox|
| Water For Elephants|              72|    Drama|2011|    20th Century Fox|
|              WALL-E|              89|Animation|2008|              Disney|
|            Waitress|              67|  Romance|2007|         Independent|
| Waiting For Forever|              53|  Romance|2011|         Independent|
|     Valentine's Day|              54|   Comedy|2010|        Warner Bros.|
|Tyler Perry

In [49]:
# mudar tipo das colunas
df_score = df_movie.withColumn('score', df_movie['Audience score %'].cast('int')).drop('Audience score %')
df_score.show()

+--------------------+---------+----+--------------------+-----+
|                Film|    Genre|Year|         Lead Studio|score|
+--------------------+---------+----+--------------------+-----+
|Zack and Miri Mak...|  Romance|2008|The Weinstein Com...|   70|
|     Youth in Revolt|   Comedy|2010|The Weinstein Com...|   52|
|You Will Meet a T...|   Comedy|2010|         Independent|   35|
|        When in Rome|   Comedy|2010|              Disney|   44|
|What Happens in V...|   Comedy|2008|                 Fox|   72|
| Water For Elephants|    Drama|2011|    20th Century Fox|   72|
|              WALL-E|Animation|2008|              Disney|   89|
|            Waitress|  Romance|2007|         Independent|   67|
| Waiting For Forever|  Romance|2011|         Independent|   53|
|     Valentine's Day|   Comedy|2010|        Warner Bros.|   54|
|Tyler Perry's Why...|  Romance|2007|         Independent|   47|
|Twilight: Breakin...|  Romance|2011|         Independent|   68|
|            Twilight|  R

In [52]:
# filtro
df_score.filter(df_score.score > 60).show()

+--------------------+---------+----+--------------------+-----+
|                Film|    Genre|Year|         Lead Studio|score|
+--------------------+---------+----+--------------------+-----+
|Zack and Miri Mak...|  Romance|2008|The Weinstein Com...|   70|
|What Happens in V...|   Comedy|2008|                 Fox|   72|
| Water For Elephants|    Drama|2011|    20th Century Fox|   72|
|              WALL-E|Animation|2008|              Disney|   89|
|            Waitress|  Romance|2007|         Independent|   67|
|Twilight: Breakin...|  Romance|2011|         Independent|   68|
|            Twilight|  Romance|2008|              Summit|   82|
|      The Ugly Truth|   Comedy|2009|         Independent|   68|
|The Twilight Saga...|    Drama|2009|              Summit|   78|
|The Time Traveler...|    Drama|2009|           Paramount|   65|
|        The Proposal|   Comedy|2009|              Disney|   74|
|         The Duchess|    Drama|2008|           Paramount|   68|
|The Curious Case ...|  F

In [54]:
# pegar o valor maximo de score
from pyspark.sql.functions import max
df_max = df_score.agg(max('score').alias('max_score')).show()

+---------+
|max_score|
+---------+
|       89|
+---------+



In [56]:
df_max_score = df_score.filter(df_score.score == 89)
df_max_score.show()

+------------------+---------+----+-----------+-----+
|              Film|    Genre|Year|Lead Studio|score|
+------------------+---------+----+-----------+-----+
|            WALL-E|Animation|2008|     Disney|   89|
|A Dangerous Method|    Drama|2011|Independent|   89|
+------------------+---------+----+-----------+-----+



In [57]:
# contar studio
from pyspark.sql.functions import count, col, asc, desc
df_sum = df_score.groupBy('Lead Studio').count()
df_sum.show()

+--------------------+-----+
|         Lead Studio|count|
+--------------------+-----+
|                Sony|    4|
|The Weinstein Com...|    3|
|           Paramount|    4|
|           Lionsgate|    2|
|              Summit|    5|
|            New Line|    1|
|              Disney|    8|
|        Warner Bros.|   14|
|           Universal|    8|
|    20th Century Fox|    2|
|                 Fox|    6|
|                 CBS|    1|
|         Independent|   19|
+--------------------+-----+



In [58]:
# ordenar contagem
df_sum.orderBy(col('count').desc()).show()

+--------------------+-----+
|         Lead Studio|count|
+--------------------+-----+
|         Independent|   19|
|        Warner Bros.|   14|
|              Disney|    8|
|           Universal|    8|
|                 Fox|    6|
|              Summit|    5|
|                Sony|    4|
|           Paramount|    4|
|The Weinstein Com...|    3|
|           Lionsgate|    2|
|    20th Century Fox|    2|
|            New Line|    1|
|                 CBS|    1|
+--------------------+-----+



In [59]:
df_sum.orderBy(col('count').asc()).show()

+--------------------+-----+
|         Lead Studio|count|
+--------------------+-----+
|            New Line|    1|
|                 CBS|    1|
|           Lionsgate|    2|
|    20th Century Fox|    2|
|The Weinstein Com...|    3|
|                Sony|    4|
|           Paramount|    4|
|              Summit|    5|
|                 Fox|    6|
|              Disney|    8|
|           Universal|    8|
|        Warner Bros.|   14|
|         Independent|   19|
+--------------------+-----+



In [65]:
# in
# filtrar os filmes que seja dos studios Universal', 'Disney' ou 'Warner Bros'
df_score.select('Lead Studio', 'Film', 'score').filter(df_score['Lead Studio'].isin('Universal', 'Disney', 'Warner Bros.')).show()

+------------+--------------------+-----+
| Lead Studio|                Film|score|
+------------+--------------------+-----+
|      Disney|        When in Rome|   44|
|      Disney|              WALL-E|   89|
|Warner Bros.|     Valentine's Day|   54|
|      Disney|        The Proposal|   74|
|Warner Bros.|The Invention of ...|   47|
|Warner Bros.|The Curious Case ...|   81|
|      Disney|             Tangled|   88|
|Warner Bros.|Sex and the City Two|   49|
|Warner Bros.|  Sex and the City 2|   49|
|Warner Bros.|    Sex and the City|   81|
|Warner Bros.|     No Reservations|   64|
|Warner Bros.|      New Year's Eve|   48|
|Warner Bros.|    Music and Lyrics|   70|
|   Universal|          Mamma Mia!|   76|
|   Universal|          Mamma Mia!|   76|
|   Universal|        Love Happens|   40|
|Warner Bros.|      License to Wed|   55|
|   Universal|           Leap Year|   49|
|   Universal|          Knocked Up|   83|
|   Universal|           Jane Eyre|   77|
+------------+--------------------

In [66]:
# not in
df_score.select('Lead Studio', 'Film', 'score').filter(~df_score['Lead Studio'].isin('Universal', 'Disney', 'Warner Bros.')).show()

+--------------------+--------------------+-----+
|         Lead Studio|                Film|score|
+--------------------+--------------------+-----+
|The Weinstein Com...|Zack and Miri Mak...|   70|
|The Weinstein Com...|     Youth in Revolt|   52|
|         Independent|You Will Meet a T...|   35|
|                 Fox|What Happens in V...|   72|
|    20th Century Fox| Water For Elephants|   72|
|         Independent|            Waitress|   67|
|         Independent| Waiting For Forever|   53|
|         Independent|Tyler Perry's Why...|   47|
|         Independent|Twilight: Breakin...|   68|
|              Summit|            Twilight|   82|
|         Independent|      The Ugly Truth|   68|
|              Summit|The Twilight Saga...|   78|
|           Paramount|The Time Traveler...|   65|
|           Paramount|  The Heartbreak Kid|   41|
|           Paramount|         The Duchess|   68|
|                 CBS|    The Back-up Plan|   47|
|         Independent|  Something Borrowed|   48|


In [74]:
# utilizar sql dentro do pyspark

df_score.createOrReplaceTempView('movies')
spark.sql('SELECT `Lead Studio`, COUNT(*) AS qtd FROM movies GROUP BY `Lead Studio` ORDER BY qtd DESC').show()


+--------------------+---+
|         Lead Studio|qtd|
+--------------------+---+
|         Independent| 19|
|        Warner Bros.| 14|
|              Disney|  8|
|           Universal|  8|
|                 Fox|  6|
|              Summit|  5|
|                Sony|  4|
|           Paramount|  4|
|The Weinstein Com...|  3|
|           Lionsgate|  2|
|    20th Century Fox|  2|
|            New Line|  1|
|                 CBS|  1|
+--------------------+---+



In [79]:
# criar df reviews
df_reviews = spark.read.csv('reviews-2.csv', header=True, inferSchema=True, sep=';')
df_reviews.show()


+--------------------+---------+--------------------+
|                Film|       ID|              Review|
+--------------------+---------+--------------------+
|Zack and Miri Mak...|100000000|Three start. I ha...|
|     Youth in Revolt|100000001|Heating and not w...|
|You Will Meet a T...|100000002|2nd hand laptops!...|
|        When in Rome|100000003|Review: BEST LAPT...|
|What Happens in V...|100000004|Review: second ha...|
| Water For Elephants|100000005|Review: very bad....|
|              WALL-E|100000006|Review: waste of ...|
|            Waitress|100000007|Review: Very bad ...|
| Waiting For Forever|100000008|Review: One star....|
|     Valentine's Day|100000009|Review: two star....|
|Tyler Perry's Why...|100000010|Review: Three sta...|
|Twilight: Breakin...|100000011|Review: Bad produ...|
|            Twilight|100000012|Review: not a goo...|
|      The Ugly Truth|100000013|Review: better la...|
|The Twilight Saga...|100000014|Review:  have had...|
|The Time Traveler...|100000

In [81]:
# inner join
df.join(df_reviews, df.Film == df_reviews.Film, 'inner')\
        .select(df['Lead Studio'], df.Film, df.Genre, df.Year, df_reviews.Review)\
        .show(truncate=False)

+---------------------+-----------------------------------+---------+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Lead Studio          |Film                               |Genre    |Year|Review                                                                                                                                                                           |
+---------------------+-----------------------------------+---------+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|The Weinstein Company|Zack and Miri Make a Porno         |Romance  |2008|Three start. I have some issues like some keys don't work on first instance..and is somewhat slow also                                                                 